## Exercises

Using the [repo setup directions](https://ds.codeup.com/fundamentals/git/), setup a new local and remote repository named `spark-exercises`. The local version of your repo should live inside of `~/codeup-data-science`. This repo should be named `spark-exercises`

Save this work in your `spark-exercises` repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named `spark101` for this exercise.



# imports 

In [1]:
import pyspark

#create spark session builder
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# 1. Create a spark data frame that contains your favorite programming languages.

    - The name of the column should be `language`
    - View the schema of the dataframe
    - Output the shape of the dataframe
    - Show the first 5 records in the dataframe



In [11]:
from pyspark.sql import Row

fav_language = spark.createDataFrame([
    Row(language='python', projects=5),
    Row(language='swift', projects=2),
    Row(language='javascript', projects=1),
    Row(language='basic', projects=1),
    Row(language='r', projects=0),
    Row(language='c++', projects=0)
    ])
fav_language

DataFrame[language: string, projects: bigint]

In [7]:
#view schema of data frame
fav_language.printSchema()

root
 |-- language: string (nullable = true)
 |-- projects: long (nullable = true)



In [9]:
#output shape of dataframe
#count gives number of rows, , len of df.columns gives number of columns
print(fav_language.count(), 'rows', len(fav_language.columns), 'columns')

6 rows 2 columns


In [13]:
#show the first 5 records in the data frame
fav_language.show(5)

+----------+--------+
|  language|projects|
+----------+--------+
|    python|       5|
|     swift|       2|
|javascript|       1|
|     basic|       1|
|         r|       0|
+----------+--------+
only showing top 5 rows



# Load the `mpg` dataset as a spark dataframe.

    * Create 1 column of output that contains a message like the one below for each vehicle:

            The 1999 audi a4 has a 4 cylinder engine.

    * Transform the `trans` column so that it only contains either `manual` or `auto`.
---


In [18]:
from pydataset import data

#import functions to use concat, sum, avg, min, max, ocount,  mean,  concat, and lit, regexpt_extract, regexp_replace
import pyspark.sql.functions as F


mpg = spark.createDataFrame(data('mpg'))

## Create 1 column of output that contains a message for each vehicle:

In [21]:
(mpg.select(F.concat(F.lit('The '), mpg.year, mpg.manufacturer, mpg.model, F.lit(' has a '), mpg.cyl, F.lit(' cylinder engine.') ))).show(5, truncate=False)

+------------------------------------------------------------------------+
|concat(The , year, manufacturer, model,  has a , cyl,  cylinder engine.)|
+------------------------------------------------------------------------+
|The 1999audia4 has a 4 cylinder engine.                                 |
|The 1999audia4 has a 4 cylinder engine.                                 |
|The 2008audia4 has a 4 cylinder engine.                                 |
|The 2008audia4 has a 4 cylinder engine.                                 |
|The 1999audia4 has a 6 cylinder engine.                                 |
+------------------------------------------------------------------------+
only showing top 5 rows



## Transform the `trans` column so that it only contains either `manual` or `auto`

In [22]:
mpg.select('trans').show(3)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
+----------+
only showing top 3 rows



In [50]:
mpg.select('trans',
    F.when(mpg.trans.contains('auto'), 'auto')
    .otherwise('manual')).show(20)

+----------+---------------------------------------------------------+
|     trans|CASE WHEN contains(trans, auto) THEN auto ELSE manual END|
+----------+---------------------------------------------------------+
|  auto(l5)|                                                     auto|
|manual(m5)|                                                   manual|
|manual(m6)|                                                   manual|
|  auto(av)|                                                     auto|
|  auto(l5)|                                                     auto|
|manual(m5)|                                                   manual|
|  auto(av)|                                                     auto|
|manual(m5)|                                                   manual|
|  auto(l5)|                                                     auto|
|manual(m6)|                                                   manual|
|  auto(s6)|                                                     auto|
|  aut

In [53]:
mpg.select('trans', F.regexp_extract('trans', r'^(\w+)', 1).alias('attempts')).show(3)

+----------+--------+
|     trans|attempts|
+----------+--------+
|  auto(l5)|    auto|
|manual(m5)|  manual|
|manual(m6)|  manual|
+----------+--------+
only showing top 3 rows



# 1. Load the `tips` dataset as a spark dataframe.

    1. What percentage of observations are smokers?
    1. Create a column that contains the tip percentage
    1. Calculate the average tip percentage for each combination of sex and smoker.



In [54]:
tips = spark.createDataFrame(data('tips'))

In [55]:
#similar to df.info
tips.printSchema()
#what is a double data type?

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: long (nullable = true)



## What Percentage of observations are smokers?

In [68]:
#tips.rollup(tips.smoker== 'Yes').count().show()
#tips.groupBy(tips.smoker).agg(F.sum(tips.smoker == 'Yes')).show() --> not good
tips.filter(tips.smoker == 'Yes').count()/tips.count()

0.38114754098360654

In [139]:
tips.groupBy('smoker').count().withColumn(
    'percent', F.round(
        F.col('count')/tips.count() *100)).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   62.0|
|   Yes|   93|   38.0|
+------+-----+-------+



## Create a column that contains the tip percentage?

In [82]:
col1 = F.round((tips.tip/tips.total_bill),2).alias('tip_percentage')
tips.select(tips.total_bill, tips.tip, col1).show(5)

+----------+----+--------------+
|total_bill| tip|tip_percentage|
+----------+----+--------------+
|     16.99|1.01|          0.06|
|     10.34|1.66|          0.16|
|     21.01| 3.5|          0.17|
|     23.68|3.31|          0.14|
|     24.59|3.61|          0.15|
+----------+----+--------------+
only showing top 5 rows



## Calculate the average tip percentage for each combination of sex and smoker.


In [145]:
tips.groupby('sex').pivot('smoker').agg(F.round(F.mean(tips.tip/tips.total_bill))).show()

+------+---+---+
|   sex| No|Yes|
+------+---+---+
|Female|0.0|0.0|
|  Male|0.0|0.0|
+------+---+---+



In [144]:
tips.withColumn('tip_percent', F.round((tips.tip/tips.total_bill), 4) * 100).show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|       tip_percent|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|              5.94|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|             16.05|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|             16.66|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|             13.98|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680000000000001|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|             18.62|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|             22.81|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|             11.61|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|             13.03|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|             21.85|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|16.650000000000002|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180000000000

# Use the seattle weather dataset referenced in the lesson to answer the questions below.

    - Convert the temperatures to fahrenheit.
    - Which month has the most rain, on average?
    - Which year was the windiest?
    - What is the most frequent type of weather in January?
    - What is the average high and low temperature on sunny days in July in 2013 and 2014?
    - What percentage of days were rainy in q3 of 2015?
    - For each year, find what percentage of days it rained (had non-zero precipitation).

In [150]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)

In [85]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



## Convert the temperatures to fahrenheit

In [151]:
weather = \
weather\
    .withColumn(
        'temp_max', (F.col('temp_max')* 9/5 +32))\
    .withColumn(
        'temp_min',(F.col('temp_min')*9/5 +32))
weather.show()

+----------+-------------+------------------+------------------+----+-------+
|      date|precipitation|          temp_max|          temp_min|wind|weather|
+----------+-------------+------------------+------------------+----+-------+
|2012-01-01|          0.0|             55.04|              41.0| 4.7|drizzle|
|2012-01-02|         10.9|             51.08|             37.04| 4.5|   rain|
|2012-01-03|          0.8|             53.06|             44.96| 2.3|   rain|
|2012-01-04|         20.3|             53.96|             42.08| 4.7|   rain|
|2012-01-05|          1.3|             48.02|             37.04| 6.1|   rain|
|2012-01-06|          2.5|             39.92|             35.96| 2.2|   rain|
|2012-01-07|          0.0|             44.96|             37.04| 2.3|   rain|
|2012-01-08|          0.0|              50.0|             37.04| 2.0|    sun|
|2012-01-09|          4.3|             48.92|              41.0| 3.4|   rain|
|2012-01-10|          1.0|42.980000000000004|             33.08|

## Which month has the most rain, on average?
* November, 5.35 inches of average rainfall

In [164]:
#using parens helps with dot notation formatting
( 
    weather.withColumn('month',F.month('date'))
    .groupBy('month')
    .agg(F.round(F.mean('precipitation'),2).alias('avg_rainfall'))
    .sort(F.col('avg_rainfall').desc())
    .show()
)

+-----+------------+
|month|avg_rainfall|
+-----+------------+
|   11|        5.35|
|   12|        5.02|
|    3|        4.89|
|   10|        4.06|
|    1|        3.76|
|    2|        3.73|
|    4|        3.13|
|    9|        1.96|
|    5|        1.67|
|    8|        1.32|
|    6|        1.11|
|    7|        0.39|
+-----+------------+



## Which year was the windiest? 
* 2012, average windpseed of 3.4

In [163]:
( 
    weather.withColumn('year',F.year('date'))
    .groupBy('year')
    .agg(F.round(F.mean('wind'),2).alias('avg_wind'))
    .sort(F.col('avg_wind').desc())
    .show()
)

+----+--------+
|year|avg_wind|
+----+--------+
|2012|     3.4|
|2014|    3.39|
|2015|    3.16|
|2013|    3.02|
+----+--------+



## What is the most frequent type of weather in January?
* Fog, 38 count


In [166]:
# narrow down the month, using F.month funciton
# aggregate weather
# aggregation function: frequency --> take the count
# sort by the count, descending

weather.filter(F.month('date')==1).groupby('weather').count().sort(F.col('count').desc()).show()
# non-viable code
# weather.select(weather.weather).filter(F.month('date')==1).show()
# (
#     weather.filter(F.month('date') == 1)
#     .groupBy('weather')
#     .agg(sum)
#     .count()
#     .show()
# )

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



## What is the average high and low temperature on sunny days in July in 2013 and 2014?


In [169]:
(
    weather.filter(F.month('date')==7)
    .where((F.year('date') == 2014) | (F.year('date') == 2013))
    .filter(weather.weather == 'sun')
    .agg(
        F.avg('temp_max'),
        F.avg('temp_min')
        )
    .show()
)

+-----------------+-----------------+
|    avg(temp_max)|    avg(temp_min)|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



## What percentage of days were rainy in q3 of 2015?


In [172]:
(
    weather.filter(F.quarter('date')==3)
    .where((F.year('date') == 2015))
    .select(F.when(F.col('weather')=='rain', 1).otherwise(0).alias('rain'))
    .agg(F.mean('rain'))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



## For each year, find what percentage of days it rained (had non-zero precipitation).

In [174]:
(
    weather.withColumn(
        'year', F.year('date')
    )
    .select(
        F.when(F.col('precipitation') > 0,1)
            .otherwise(0).alias('did_rain'), 'year'
        )
    .groupby('year')
    .agg(F.mean('did_rain'))``
    .show()
)


+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

